In [7]:
import ujson as json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fasttext
from pycountry import languages
import re
import json

In [2]:
PRETRAINED_MODEL_PATH = 'lid.176.bin'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [3]:
# function to help figure out the language
# adjusted to check if the body is a float (nan)
def lang_class(x):
    # little bit of error handling in case the darn thing isn't working the way I'd like
    if type(x) != float:
        x = x .replace("\n", " ")
        lang = model.predict(x)
        
        return lang

In [4]:
# function to help retrieve the language name using the language code
def lang_from_code(x):
    x = x[0]
    x = str(x)
  
    match = re.match(r"__label__([a-zA-Z]+)", x)
    match = match.group(1)

    if len(match) == 2:
        
        try:
            lang = languages.get(alpha_2=match).name

        except:
            lang = match
    elif len(match) == 3:
        try:
            lang = languages.get(alpha_3=match).name
        
        except:   
            lang = match
            
        
    return(lang)

In [5]:
# function to detect the languages, apply those as labels, and then subset on that
# function gets passed the whole dataframe
def lang_filt(df):
    # making the lists of cols to drop & keep
    to_keep = ["t","id","cd","c","u","un", "dmn", "bo"]
    to_drop = [x for x in list(df.columns) if x not in to_keep]
        
    # dropping the cols
    df = df.drop(to_drop, axis = 1)
     
     # filtering to those which have a body (and can have a language classifier run on them)
    df = df[df["bo"].notna()]
    
    # applying the lang_class func
    df["bo_lang"] = df["bo"].apply(lang_class)
       
    # making the list of labels and confidences
    df[["bo_lang","conf"]] = pd.DataFrame(df[df["bo_lang"].notna()]["bo_lang"].tolist(), index=df[df["bo"].notna()].index)
    
    # pulling the language codes
    df["bo_lang"] = df[df["bo_lang"].notna()]["bo_lang"].apply(lang_from_code)
    
    return df

In [6]:
# kinda messy -> can clean this up so it's prettier
# function to pull out the top languages in there before filtering
# function gets passed whole dataframe
def top_langs(df, lang_dict, top_n):    
    
    # starting with an empty dict for overall
    lang_dict.setdefault("overall", {})
                
    # following up by converting counts and percents to a dictionary & appending those
    lang_dict["overall"].setdefault("counts",
                                df[df["bo_lang"].notna()]["bo_lang"].value_counts().head(top_n).to_dict())
    
    lang_dict["overall"].setdefault("percents",
                                df[df["bo_lang"].notna()]["bo_lang"].value_counts(normalize=True).head(top_n).to_dict())

    # starting with an empty dict for the posts
    lang_dict.setdefault("posts", {})
        
    # following up by converting counts and percents to a dictionary & appending those
    lang_dict["posts"].setdefault("counts",
                        df[(df["bo_lang"].notna()) & (df["t"] == "p")]["bo_lang"].value_counts().head(top_n).to_dict())
    
    lang_dict["posts"].setdefault("percents",
        df[(df["bo_lang"].notna()) & (df["t"] == "p")]["bo_lang"].value_counts(normalize=True).head(top_n).to_dict())

    # starting with an empty dict for the comments
    lang_dict.setdefault("comments", {})
        
    # following up by converting counts and percents to a dictionary & appending those
    lang_dict["comments"].setdefault("counts",
                        df[(df["bo_lang"].notna()) & (df["t"] == "c")]["bo_lang"].value_counts().head(top_n).to_dict())
    
    lang_dict["comments"].setdefault("percents",
        df[(df["bo_lang"].notna()) & (df["t"] == "c")]["bo_lang"].value_counts(normalize=True).head(top_n).to_dict())
    
    # returns the lagnuage dict we passed it with those adjustments
    return lang_dict

In [11]:
# function takes in file name and gives a new one for the new save file we're making
def file_name(file):
    # splitting on "."
    file = file.split(".")
    # adding the name to show it's been filtered
    file[0] = file[0] + "_lang_filtered"
    # ading the file type
    file = file[0] + "." + file[1]
    # returning the file
    return file

In [ ]:
# deprecated code
# print("\nOverall")
# # printing the counts and percents overall
# print(pd.concat([df[df["bo_lang"].notna()]["bo_lang"].value_counts().head(top_n), 
#       df[df["bo_lang"].notna()]["bo_lang"].value_counts(normalize=True).head(top_n)], axis=1, keys=["count", "pct"]))

# print("\nPosts")
# # printing counts & percents for posts
# print(pd.concat([df[(df["bo_lang"].notna()) & (df["t"] == "p")]["bo_lang"].value_counts().head(top_n), 
#      df[(df["bo_lang"].notna()) & (df["t"] == "p")]["bo_lang"].value_counts(normalize=True).head(top_n)],
#         axis=1, keys=["count", "pct"]))

# print("\nComments")
# # printing counts & percents for comments
# print(pd.concat([df[(df["bo_lang"].notna()) & (df["t"] == "c")]["bo_lang"].value_counts().head(top_n), 
#      df[(df["bo_lang"].notna()) & (df["t"] == "c")]["bo_lang"].value_counts(normalize=True).head(top_n)],
#         axis=1, keys=["count", "pct"]))    

In [ ]:
# still need to change the lange filtering to follow the same approach I used in the domain filtering
# to whit, I need to output top n results to a dictionary

In [29]:
### this is a test!!!
# this is meant to output a dictionary that I can then turn into an ndjson file

In [12]:
%%time
# needs to run through each of the datasets -> they need to be read in
lang_dict = {}

for file in file_list:
    # reading file   
    df = pd.read_json(file, lines=True)
    
           
    # filtering to english language only & pulling the languages prior to filtering 
    # (those languages are post empty bo rm)
    df = lang_filt(df)
    
    # declaring empty dict
    lang_dict.setdefault(file, {})

    # running on the top 10 languages
    lang_dict[file] = top_langs(df, lang_dict[file], 10)

    
    # filtering the language down to just english
    df = df[df["bo_lang"] == "English"]
    
    # possible
    
    
    # creating the file name
    filename = file_name(file)
    
    
    
    # exporting the file
    df.to_json(filename, orient="records",lines=True)
    


NameError: name 'coh_perp_dict' is not defined

In [13]:
# serializing the lang_dict
# saving the the test run coherence and perplexity for each of the models
with open('lang_json', 'w') as outfile:
    json.dump(lang_dict, outfile)

In [74]:
df.head()

,t,id,cd,c,u,un,dmn,bo,bo_lang,conf
1,p,3619348b91524430882f2b887838a3e4,20201018,1602989910000,8b67993183a14587a001010058d089d2,chucknellis,noqreport.com,Facebook’s Public Policy Manager for Global El...,English,[0.8107684254646301]
2,p,1365051bfe6243599e6af1055b71c4a2,20201202,1606897009000,eeeb8dd25b7142b1bc69cbdbe1d8bb62,ThomasFox,rumble.com,"USPS Driver Drops SHOCKING Claim About 200,000...",English,[0.6809220314025879]
5,p,d755c2ce09e74c9b9b34953aae839554,20190829,1567070618000,21031f424913456591d9a9aed4ff26c7,Cobrarick98,i.imgur.com,Antifa. You touch a Trump supporter we will be...,English,[0.8768866658210754]
7,p,a0124e3f9afa4fe29d404e41e3ac9a5f,20201009,1602262587000,6e6d4f8e7479446f8f06d5d5c0fae9a3,AppleJax,image-cdn.parler.com,If the guilty aren't charged before the electi...,English,[0.9819905757904053]
9,p,857c1349a1cc4bfba84cb75a9a34ec93,20201026,1603746735000,eeeb8dd25b7142b1bc69cbdbe1d8bb62,ThomasFox,deadline.com,‘60 Minutes’ & Trump Walk-Off Soars To 17M Vie...,English,[0.7728871703147888]


#### Detecting the languages and adding that as a class filter

In [9]:
file_list = ["min_act.ndjson", "mod_act.ndjson", "ver_act.ndjson","ext_act.ndjson"]

In [8]:
for file in file_list:
    file = file.split(".")
    file[0] = file[0] + "_lang_filtered"
    file = file[0] + "." + file[1]
    print(file)

min_act_lang_filtered.ndjson
mod_act_lang_filtered.ndjson
ver_act_lang_filtered.ndjson
ext_act_lang_filtered.ndjson


In [11]:
%%time
for file in file_list:
    # reading file
    df = pd.read_json(file, lines=True)
    
    # printing out the current dataset we're working on
    print("Current Dataset is:", file)
    print("Language information for file:")
    
    # filtering to english language only
    df = lang_filt(df)
    
    filename = file_name(file)
    
    # exporting the file
    df.to_json(filename, orient="records",lines=True)

Current Dataset is: min_act.ndjson
Language information for file:

Overall
               count       pct
English     21506207  0.911404
Portuguese    332572  0.014094
Arabic        250503  0.010616
German        223956  0.009491
Spanish       178334  0.007558
Japanese      177927  0.007540
Russian       114725  0.004862
Chinese       109535  0.004642
Italian       103426  0.004383
French         88103  0.003734

Posts
              count       pct
English     5319874  0.912323
Arabic       124498  0.021351
Portuguese   106613  0.018283
Spanish       42037  0.007209
German        29656  0.005086
Japanese      29480  0.005056
Persian       28209  0.004838
Chinese       18808  0.003225
French        18007  0.003088
Italian       17841  0.003060

Comments
               count       pct
English     16186333  0.911103
Portuguese    225959  0.012719
German        194300  0.010937
Japanese      148447  0.008356
Spanish       136297  0.007672
Arabic        126005  0.007093
Russian       102958

In [19]:
%%time
df = pd.read_json("ext_act.ndjson", lines=True)

CPU times: total: 25.2 s
Wall time: 33.8 s


In [ ]:
# step 1) rm those without bodeies
# step 2) get lang_counts
# step 3) filter eng only

,t,id,cd,c,u,un,dmn,bo
1,p,3619348b91524430882f2b887838a3e4,20201018,1602989910000,8b67993183a14587a001010058d089d2,chucknellis,noqreport.com,Facebook’s Public Policy Manager for Global El...
2,p,1365051bfe6243599e6af1055b71c4a2,20201202,1606897009000,eeeb8dd25b7142b1bc69cbdbe1d8bb62,ThomasFox,rumble.com,"USPS Driver Drops SHOCKING Claim About 200,000..."
5,p,d755c2ce09e74c9b9b34953aae839554,20190829,1567070618000,21031f424913456591d9a9aed4ff26c7,Cobrarick98,i.imgur.com,Antifa. You touch a Trump supporter we will be...
7,p,a0124e3f9afa4fe29d404e41e3ac9a5f,20201009,1602262587000,6e6d4f8e7479446f8f06d5d5c0fae9a3,AppleJax,image-cdn.parler.com,If the guilty aren't charged before the electi...
9,p,857c1349a1cc4bfba84cb75a9a34ec93,20201026,1603746735000,eeeb8dd25b7142b1bc69cbdbe1d8bb62,ThomasFox,deadline.com,‘60 Minutes’ & Trump Walk-Off Soars To 17M Vie...
...,...,...,...,...,...,...,...,...
2254410,c,ae0a1d21bc7144a7a024c90dcf7c1a5d,20200723,1595479941000,f8b37b14d52f493eab60c60486d5e6c6,Aj118,image-cdn.parler.com,@Salonsbocaraton
2254416,c,dc3fa8b7c12c4e3c898af4b14c99cbf5,20200727,1595810868000,6e6d4f8e7479446f8f06d5d5c0fae9a3,AppleJax,image-cdn.parler.com,Yeah I know it’s bullshit but it’s also racist...
2254419,c,f2b26de4201e4519879b63d4b1e0eea7,20200824,1598305363000,f8b37b14d52f493eab60c60486d5e6c6,Aj118,image-cdn.parler.com,@Buellh
2254423,c,1dc1abde35234e32a872bcf76bd6ab45,20200820,1597891420000,efaac1ece6a893e0fab48f558e3a5c57,BethocAeilflaed,image-cdn.parler.com,This…\n\nFull of wine…\n\nOnly thing keeping m...


In [157]:
%%time
df = lang_filt(df)

test
Overall
              count       pct
English     1266682  0.916547
Portuguese    25279  0.018291
Japanese      21815  0.015785
Spanish       12620  0.009132
German        10262  0.007425
Chinese        8512  0.006159
French         6606  0.004780
Catalan        4367  0.003160
Russian        4361  0.003156
Italian        3756  0.002718

Posts
              count       pct
English     1061052  0.933251
Portuguese    24135  0.021228
Japanese      13695  0.012045
Spanish        7227  0.006357
German         5597  0.004923
Chinese        5029  0.004423
French         3852  0.003388
Catalan        3815  0.003355
Russian        2231  0.001962
Italian        1557  0.001369

Comments
             count       pct
English     205630  0.839056
Japanese      8120  0.033133
Spanish       5393  0.022006
German        4665  0.019035
Chinese       3483  0.014212
French        2754  0.011237
Italian       2199  0.008973
Russian       2130  0.008691
Portuguese    1144  0.004668
Esperanto      938  

In [147]:
df

,t,id,cd,c,u,un,dmn,prov,bo,bo_lang,conf
0,p,50f27e734766438698d4d5c2d689a5f1,20201109,1604927870000,87be30dacdc74cd2936bcbe4ece6010d,Bullscricker,image-cdn.parler.com,post,NaN,NaN,NaN
1,p,3619348b91524430882f2b887838a3e4,20201018,1602989910000,8b67993183a14587a001010058d089d2,chucknellis,noqreport.com,post,Facebook’s Public Policy Manager for Global El...,English,[0.8107684254646301]
2,p,1365051bfe6243599e6af1055b71c4a2,20201202,1606897009000,eeeb8dd25b7142b1bc69cbdbe1d8bb62,ThomasFox,rumble.com,post,"USPS Driver Drops SHOCKING Claim About 200,000...",English,[0.6809220314025879]
3,p,5f5e02a226454cbb9f56741a3aa6adce,20200824,1598283523000,e04afa3752f94366813196fb88022440,Kreermary,image-cdn.parler.com,post,NaN,NaN,NaN
4,p,3e689379be5b4e1fa333bbf7f5e1fad2,20201219,1608385852000,46563f08c3a34bebbf4960877d77230f,Starblazer692003,image-cdn.parler.com,post,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2254421,c,1ff3e15913c34456828bc70a7966bda1,20201009,1602255581000,e11890c2ffc348df9cb703651a3cc9c0,LibertyElaine,image-cdn.parler.com,comment,NaN,NaN,NaN
2254422,c,0a94034622254a0f96f943a2f7b98621,20200829,1598718069000,f8b37b14d52f493eab60c60486d5e6c6,Aj118,image-cdn.parler.com,comment,NaN,NaN,NaN
2254423,c,1dc1abde35234e32a872bcf76bd6ab45,20200820,1597891420000,efaac1ece6a893e0fab48f558e3a5c57,BethocAeilflaed,image-cdn.parler.com,comment,This…\n\nFull of wine…\n\nOnly thing keeping m...,English,[0.6988052725791931]
2254424,c,4b80d4aa6f9d4775a8ba82f940272507,20200817,1597632198000,f8b37b14d52f493eab60c60486d5e6c6,Aj118,image-cdn.parler.com,comment,NaN,NaN,NaN
